In [20]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data.sampler import SubsetRandomSampler

import torchvision
from torchvision import transforms
import torchvision.models as models

from time import time
import logging
import numpy as np
import pandas as pd
import os

from torch.nn import ReLU
import matplotlib.pyplot as plt

In [18]:
#setting up all the parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

kwargs = {'num_workers': 1, 'pin_memory': True, batch_size = 128} if torch.cuda.is_available() else {}
print(kwargs)

DATA_PATH = r"C:\Users\josep\Dropbox\Joseph\AgroParisTech\IODAAAAA\Projet Fil Rouge\PFR_Cirses_2022_local_run\data"
print(DATA_PATH)

cuda
{'num_workers': 1, 'pin_memory': True}
C:\Users\josep\Dropbox\Joseph\AgroParisTech\IODAAAAA\Projet Fil Rouge\PFR_Cirses_2022_local_run\data


In [22]:
#benchmarking the num workers 

import multiprocessing as mp

for num_workers in range(2, mp.cpu_count(), 2):  
    train_loader = DataLoader(DATA_PATH,shuffle=True, num_workers=num_workers ,batch_size=128,pin_memory=True)
    start = time()
    for epoch in range(1, 3):
        for i, data in enumerate(train_loader, 0):
            pass
    end = time()
    print("Finish with:{} second, num_workers={}".format(end - start, num_workers))

Finish with:2.838589906692505 second, num_workers=2
Finish with:3.088090658187866 second, num_workers=4
Finish with:3.6544947624206543 second, num_workers=6
Finish with:4.4289634227752686 second, num_workers=8
Finish with:5.138800382614136 second, num_workers=10


In [23]:
class DataSplit:
    """
    Replace DataLoader !
    Allows to triple Load the data into three differents sets : train, test and val
    The size of the datasets depends on the splits inputed as train_split, val_split and test_split
    It is possible to have the data randomly sampled with the argument shuffle = True
    """
    def __init__(self, dataset, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=False):
        """
        Initialize the data split based on the splits given in inputs
        Inputs : 
            - dataset : dataset loaded with torchivison.datasets.ImageFolder()
            - train_split, val_split and test_split : the size of the splits in % of the dataset size
            - shuffle : the possibility of sampling randomly through the dataset
        """
        self.batch_size=128
        self.num_workers=2
        
        self.dataset = dataset
        dataset_size = len(dataset)
        self.indices = list(range(dataset_size)) #all the indices
 
        #the quantity of data in each section of dataset
        train_split = int(np.floor(train_split * dataset_size))
        val_split = int(np.floor(val_split * dataset_size))
        #test split is what is left not sampled
        test_split = int(np.floor(dataset_size - train_split - val_split))
    
        if shuffle:
            np.random.shuffle(self.indices)
        
        self.train_indices = self.indices[:train_split]
        self.val_indices = self.indices[train_split : train_split+val_split]
        self.test_indices = self.indices[train_split + val_split :]

        self.train_sampler = SubsetRandomSampler(self.train_indices)
        self.val_sampler = SubsetRandomSampler(self.val_indices)
        self.test_sampler = SubsetRandomSampler(self.test_indices)

    def get_train_split_point(self):
        return len(self.train_sampler) + len(self.val_indices)


    def get_validation_split_point(self):
        return len(self.train_sampler)


    def get_split(self):
        print('Initializing train-validation-test dataloaders')
        self.train_loader = self.get_train_loader(batch_size=self.batch_size, num_workers=self.num_workers)
        self.val_loader = self.get_validation_loader(batch_size=self.batch_size, num_workers=self.num_workers)
        self.test_loader = self.get_test_loader(batch_size=self.batch_size, num_workers=self.num_workers)
        return self.train_loader, self.val_loader, self.test_loader
    

    def get_train_loader(self, batch_size=50, num_workers=4):
        logging.debug('Initializing train dataloader')
        self.train_loader = torch.utils.data.DataLoader(self.dataset, sampler=self.train_sampler, shuffle=False, **kwargs)
        return self.train_loader


    def get_validation_loader(self, batch_size=50, num_workers=4):
        logging.debug('Initializing validation dataloader')
        self.val_loader = torch.utils.data.DataLoader(self.dataset, sampler=self.val_sampler, shuffle=False, **kwargs)
        return self.val_loader


    def get_test_loader(self, batch_size=50, num_workers=4):
        logging.debug('Initializing test dataloader')
        self.test_loader = torch.utils.data.DataLoader(self.dataset, sampler=self.test_sampler, shuffle=False, **kwargs)
        return self.test_loader

In [32]:
normal_images = torchvision.datasets.ImageFolder(root=DATA_PATH, transform=transforms.ToTensor())
B = DataSplit(normal_images, shuffle=True)

# Loading the data into three different sets
train = B.get_train_loader()
# print(f"is train on GPU ? {train.is_cuda}" ) doesn't work
test = B.get_test_loader()
val = B.get_validation_loader()

AttributeError: 'DataLoader' object has no attribute 'is_cuda'

In [27]:
#loading the three models 
mod = models.resnet18(pretrained=True).to(device=device)
num_ftrs = mod.fc.in_features
mod.fc = nn.Linear(num_ftrs, 2)

mod_green = models.resnet18(pretrained=True).to(device=device)
num_ftrs_green = mod_green.fc.in_features
mod_green.fc = nn.Linear(num_ftrs, 2)

mod_skelet = models.resnet18(pretrained=True).to(device=device)
num_ftrs_skelet = mod_skelet.fc.in_features
mod_skelet.fc = nn.Linear(num_ftrs, 2)

In [28]:
#la var str est le nom qu'on atribue les données générés par la pipeline
#contiennent cette var afin de les retrouver
normal = ("normal", mod, train, test, val)

#permet d'éviter de rentrer des tuples à la main
#Avec une boucle for on passe on passe automatiquement
#les tuples dans la pipeline
features = [normal]

# Pipeline d'entrainement du modèle

In [29]:
class our_pipeline:
    def __init__(self,feature, num_epochs:int, early_stopping_threshold:int):
        
        self.net = feature[0]
        self.model = feature[1]
        self.train = feature[2]
        self.val = feature[3]
        self.test = feature[4]

        self.num_epochs = num_epochs
        self.early_stopping_threshold = early_stopping_threshold
        self.nb_class = len(self.train.dataset.classes)

        self.optimizer = Adam(self.model.parameters())
        #self.scheduler = CyclicLR(self.optimizer, base_lr=0.001, max_lr=0.01, step_size=10, mode='triangular')
        self.criterion = nn.CrossEntropyLoss()
        self.scheduler = StepLR(self.optimizer, step_size=10)

    def performances(self, data):

        #les métriques déjà programmées ne conviennent pas on crée les notres
        #Calcul de la sensibilité et de la spécificité
        nb_correct_pred = [0] * self.nb_class
        nb_total_pred = [0] * self.nb_class

        with torch.no_grad():
            for inputs, labels in data:
                outputs = self.model(inputs)
                #Sorti du model liste de proba d'appartenir à une classe
                #pred = argmax de la liste
                _, predicted = torch.max(outputs.data, 1)

                for i in range(len(labels)):
                    if labels[i] == predicted[i]:
                        nb_correct_pred[labels[i]] += 1
                    nb_total_pred[labels[i]] += 1
                
            self.good_pred = [nb_correct_pred[i] / nb_total_pred[i] for i in range(self.nb_class)]
        return self.good_pred

    def training(self):
        self.model.train()
        for inputs, labels in self.train:
            #réinitialisation du gradient
            self.optimizer.zero_grad()
            outputs = self.model(inputs)
            loss = self.criterion(outputs, labels)
            loss.backward()
            self.optimizer.step()

    def row(self, epoch):
        #Pour chaque epoch on veut avoir l'historique des performances sur train
        #et sur val ==> 1 ligne de data frame = perf d'1 epoch
        train_perf = self.performances(self.train)
        val_perf = self.performances(self.val)

        # balanced acc = (sepecificité + sensibilité)/2
        self.bal_acc_val = val_perf[0]+val_perf[1]/self.nb_class
        self.bal_acc_train = train_perf[0]+train_perf[1]/self.nb_class

        self.newrow = pd.DataFrame({'mod': [self.net], 'epoch': [epoch],
                            'true_neg_train':[train_perf[0]], 'true_pos_train':[train_perf[1]],
                            'true_neg_val':[val_perf[0]], 'true_pos_val':[val_perf[1]],
                            'balanced_acc_train': [self.bal_acc_train],
                            'balanced_acc_val': [self.bal_acc_val]
                            },index=[0])
        
        return self.newrow, self.bal_acc_val
        # on renvoi self.bal_acc_val car on s'en sert pour arrêter l'entrainement
        #En cas de surapprentissage

    def training_loop(self, print_res : bool):

        counter = 0
        best_acc = 0.0

        col_names = ['mod', 'epoch', 'true_neg_train', 'true_pos_train',
                        'true_neg_val', 'true_pos_val', 'balanced_acc_train',
                        'balanced_acc_val']

        stats = pd.DataFrame(columns=col_names)

        for epoch in range(self.num_epochs):
            self.training()
            perfs = self.row(epoch)
            if print_res : 
                print("state", self.net, ": ", epoch, "/", self.num_epochs)
                print(perfs[0])
            #A chaque époque on ajoute les stats
            stats = pd.concat([stats, perfs[0]])

            if perfs[1] > best_acc:
                best_acc = perfs[1]
                best_mod = self.model.state_dict()
                counter = 0
            else:
                counter += 1

            if counter >= self.early_stopping_threshold:
                if print_res : 
                    print("Early stopping at epoch: ", epoch+1)
                break
        #Sauvegarde du model avec l'heure
        now = time.time()
        mod_save_path = f"/work/{now}-Models_history/" + self.net + ".pt"
        torch.save(best_mod, mod_save_path)

        #Sauvegarde de l'historique avec l'heure
        stats_save_path = "/work/{now}-Models_history/" + self.net + ".csv"
        stats.to_csv(stats_save_path, index=False)

In [31]:
for feature in tqdm (features, total = len(features)) :
    C = our_pipeline(feature, 200, 20)
    C.training_loop(print_res = True)

  0%|                                                                                            | 0/1 [00:01<?, ?it/s]


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

# Évaluation qualitative du modèle

In [ ]:
def getSaliency(model,img,label):
    model.eval()
    #img = img.to(device)
    img.requires_grad = True    # ajout du gradient sur l'image elle-même
                                # idée: + gd gradient = + fort impact sur la décision
    img.grad = None
    outputs = nn.Softmax(dim=1)(model(img.unsqueeze(0)))    # ATTENTION, le softmax est dans la loss, pas dans le réseau 
                                                            # => il faut le remettre à la main pour que ça marche
    _, predicted = torch.max(outputs.data, 1)
    output=outputs[0,label]     # focalisation sur la vérité terrain
    output.backward()           # calcul des gradients associés à cette classe seulement
    sal=img.grad.abs()          # récupération du gradient sur l'image (+abs)
    if sal.dim()>2:
        sal=torch.max(sal,dim=0)[0]
    
    print("True label :", label.item(), "Predicted label : ", predicted.item())
    fig=plt.figure(figsize=(8, 8))
    fig.add_subplot(1, 2, 1)
    plt.imshow(img.detach().cpu().permute(1,2,0),cmap="gray")

    fig.add_subplot(1, 2, 2)
    plt.imshow(sal.to('cpu'),cmap="seismic",interpolation="bilinear")
    
    plt.show()

In [ ]:
model = models.resnet18()
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
state_dict = torch.load("/work/Models_history/normal.pt")
model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
nb = 0
while nb < 20:
    for inputs, labels in val:
        for i in range(len(labels)):
            getSaliency(model,inputs[i],labels[i])
            nb += 1
            if nb == 20:
                break
        if nb == 20:
            break

RuntimeError: number of dims don't match in permute

# Reconstitution de l'image

In [6]:
def global_evaluation (img_path:str, model):

    #Division de la grande image en sous-images
    sushi = SubimageCreator(img_path, size=(100, 100))
    sushi.cut()

    #Chaque sous-image passe par le model pour être classifiée
    for subimage in sushi.subimages:
        model.eval()
        outputs = nn.Softmax(dim=1)(model(img.unsqueeze(0)))
        _, prediction = torch.max(outputs.data, 1)
        
        #S'il y a un cirse sur l'imagette
        #on la colore en rouge, sinon
        #on la colore en vert
        if prediction == 0:
            col = 60
        else:
            col = 0
            
        path=sushi.output_dir+"/"+subimage
        img = cv2.imread(path)
        image_hued=change_hue(img,randint(10,col))
        cv2.imwrite(sushi.output_dir+"/"+"hued_"+subimage,image_hued)
    sushi.rebuild("hued_")

In [7]:
path = "DJI_0202.JPG"
global_evaluation(path, model)

NameError: name 'model' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6f211ba4-8439-4b4f-a72d-48759d094385' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>